In [1]:
import numpy as np
import networkx as nx
import time
import read_instances as ri
from numba import jit

In [2]:
def create_model(path):
    """
        Create Graph representation and MDP variables with number of states designated
        path: path of instance
    """
    data_format = path.split('/')[1]
    
    if data_format == 'edge_explicity':
        return ri.read_edge_explicity_instances(path)

    elif data_format == 'node_coordinate':
        return ri.read_node_coordinate_instances(path)
    else:
        raise Exception

In [3]:
@jit(nopython=True)
def update_adj_mat(tour):
    '''
        Adjacency matrix to nn algorithm purposes
        tour: partial ou complete tour to create adjacency matrix, ex:
        [0, 2, 4]  < for 5 cities [0, 1, 2, 3, 4, 5] and starting city is 0 
        neighbor with travel cost equal inf is never a good option, 
        so cities in tour recieve this travel cost.
        return: adjacency matrix
    '''
    # Adjacency matrix
    #adj_mat = np.array(nx.adjacency_matrix(G, weight='w').todense(), dtype=float)
    adj_mat = ADJ_MAT.copy()
    # set main diagonal to inf
    for i in range(NUM_CITIES): adj_mat[i][i] = np.inf
    # set elements of current tour to inf
    for i in range(NUM_CITIES):
        for t in tour[:-1]:
            adj_mat[i][t] = np.inf
    return adj_mat

In [4]:
@jit(nopython=True)
def calculate_cost(tour):
    '''
        Calc cost of tour.
        tour: tour of traveler, ex:
        [0, 2, 4, 1, 3, 0]  <- for 5 cities and starting city is 0 
        return: cost of tour
    '''
    cost = 0.
    for i in range(NUM_CITIES):
        cost += ADJ_MAT[tour[i]][tour[i+1]]
    return cost

In [5]:
@jit(nopython=True)
def nearest_neighbor(tour):
    '''
        Base heuristic to rollout algorithm, the main difference of traditional 
        nearest neighbor algorithm is the beginning  with a partial tour (solution), ex:
        tour is list of nodes [0, 1, 2], for 5 cities and starting city is 0,
        the return is a complete tour, in the form [0, 1, 2, 4, 3, 0] and cost of tour 
        
    '''
    adj_mat = update_adj_mat(tour)
    # Complete tour 
    for _ in range(NUM_CITIES - len(tour)):
        min_index = np.argmin(adj_mat[tour[-1]])
        for t in tour:
            adj_mat[min_index][t] = np.inf
            adj_mat[t][min_index] = np.inf
        tour.append(min_index)
    tour.append(tour[0])
    # Return complete tour
    return tour

In [6]:
def rollout_algorithm(G, starting_node):
    tour = [starting_node]

    for i in range(NUM_CITIES - 1):
        current_tour = tour.copy()
        best_rollout_cost = np.inf
        best_next_city = None
        
        for j in set(G[tour[-1]]) - set(tour):
            current_tour.append(j)

            nn_path = nearest_neighbor(current_tour.copy())
            rollout_cost = calculate_cost(nn_path)
            if rollout_cost < best_rollout_cost:
                best_rollout_cost = rollout_cost
                best_next_city = j
            
            current_tour.pop()
        tour.append(best_next_city)
    tour.append(starting_node)

    return tour, calculate_cost(tour)

In [8]:
def experiments_with(file_path):
    global ADJ_MAT
    global NUM_CITIES
    
    G = create_model(file_path)
    ADJ_MAT = np.array(nx.adjacency_matrix(G, weight='w').todense(), dtype=float)
    NUM_CITIES = G.number_of_nodes()

    starting_node = int(np.random.choice(range(NUM_CITIES)))

    start = time.time()
    rollout_tour, rollout_cost = rollout_algorithm(G, starting_node)
    rollout_time = time.time() - start
    
    start = time.time()
    nn_tour = nearest_neighbor([starting_node])
    nn_time = time.time() - start
    print('Custo do rollout tour: ', rollout_cost)
    print('Tempo do rollout algorithm: ', rollout_time)
    print('Custo do nn tour: ', calculate_cost(nn_tour))
    print('Tempo do nn algorithm: ', nn_time)

experiments_with('instances/edge_explicity/brazil58.tsp')

Custo do rollout tour:  27239.0
Tempo do rollout algorithm:  1.3422954082489014
Custo do nn tour:  32231.0
Tempo do nn algorithm:  0.0
